In [5]:
!jupyter nbconvert --to script repair.ipynb
from repair import *

[NbConvertApp] Converting notebook repair.ipynb to script
[NbConvertApp] Writing 10386 bytes to repair.py


In [7]:
def side_by_side(lines1, lines2, gap=4):
    max_len = max(len(lines1), len(lines2))
    lines1 += [''] * (max_len - len(lines1))
    lines2 += [''] * (max_len - len(lines2))
    return [f"{a:<60}{' ' * gap}{b}" for a, b in zip(lines1, lines2)]

def compare_structures(obj1, obj2):
    eq = True
    if isinstance(obj1, list) and isinstance(obj2, list):
        if len(obj1) != len(obj2):
            print(f"List lengths differ: {len(obj1)} vs {len(obj2)}")
            eq = False
        for i, (a, b) in enumerate(zip(obj1, obj2)):
            if a != b:
                eq = False
                print(f"List elements differ at index {i}:\n  Left:  {a}\n  Right: {b}")
        if len(obj1) != len(obj2):
            extra = obj1[len(obj2):] if len(obj1) > len(obj2) else obj2[len(obj1):]
            eq = False
            print(f"Extra elements in {'left' if len(obj1) > len(obj2) else 'right'} list: {extra}")

    elif isinstance(obj1, dict) and isinstance(obj2, dict):
        keys1 = set(obj1.keys())
        keys2 = set(obj2.keys())
        all_keys = keys1 | keys2
        for key in all_keys:
            a = obj1.get(key, None)
            b = obj2.get(key, None)
            if a != b:
                eq = False
                print(f"Dict entries differ for key {key}:\n  Left:  {a}\n  Right: {b}")
    else:
        eq = False
        print("Unsupported types for comparison. Use only lists or dicts.")
    if eq:
        print("OK")


def debug(string, k_tuple, new_symbol, ab, aa, sb, sa):
    sequence, active_k_tuples = construct_active_k_tuples_and_sequence(string_to_symbol_list(string)[0], 2)
    priority_queue = construct_priority_queue(active_k_tuples)

    print("active_k_tuples before")
    compare_structures(active_k_tuples, ab)
    print("sequence before")
    compare_structures(sequence, sb)
    
    s_active_before = KTupleInfo.get_print_lines(active_k_tuples)
    s_seq_before = SequenceElement.get_print_lines(sequence)

    replace_active_k_tuple(priority_queue, active_k_tuples, sequence, k_tuple, new_symbol, 2)

    s_active_after = KTupleInfo.get_print_lines(active_k_tuples)
    s_seq_after = SequenceElement.get_print_lines(sequence)

    print("active_k_tuples after")
    compare_structures(active_k_tuples, aa)
    print("sequence after")
    compare_structures(sequence, sa)

    print("\n" + "-" * 100)
    print("Active K-Tuples (before vs after)")
    for line in side_by_side(s_active_before, s_active_after):
        print(line)

    print("\n" + "-" * 100)
    print("Sequence (before vs after)")
    for line in side_by_side(s_seq_before, s_seq_after):
        print(line)
    print("-" * 100)

In [11]:
active_k_tuples_before = {
    (0, 1): KTupleInfo(count=2, last=3, pos_in_queue=0),
    (1, 0): KTupleInfo(count=1, last=2, pos_in_queue=0),
}

active_k_tuples_after = {
    (0, 1): KTupleInfo(count=0, last=-1, pos_in_queue=0),
    (1, 0): KTupleInfo(count=0, last=-1, pos_in_queue=0),
    (0, 3): KTupleInfo(count=1, last=2, pos_in_queue=-1),
    (3, 1): KTupleInfo(count=1, last=3, pos_in_queue=-1),
}

sequence_before = [
    SequenceElement(symbol=0, pos=0, prev_k_tuple=-1, next_k_tuple=-1),
    SequenceElement(symbol=1, pos=1, prev_k_tuple=-1, next_k_tuple=3),
    SequenceElement(symbol=0, pos=2, prev_k_tuple=-1, next_k_tuple=-1),
    SequenceElement(symbol=1, pos=3, prev_k_tuple=1, next_k_tuple=-1),
]

sequence_after = [
    SequenceElement(symbol=0, pos=0, prev_k_tuple=-1, next_k_tuple=-1),
    SequenceElement(symbol=-1, pos=1, prev_k_tuple=0, next_k_tuple=2),
    SequenceElement(symbol=3, pos=2, prev_k_tuple=-1, next_k_tuple=-1),
    SequenceElement(symbol=1, pos=3, prev_k_tuple=-1, next_k_tuple=-1),
]


debug("0101", (1,0), 3, active_k_tuples_before, active_k_tuples_after, sequence_before, sequence_after)
print("-------------------------------------------------------------------------------------------")
print("                                          OK                                               ")
print("-------------------------------------------------------------------------------------------")


active_k_tuples before
OK
sequence before
OK
active_k_tuples after
OK
sequence after
OK

----------------------------------------------------------------------------------------------------
Active K-Tuples (before vs after)
Key    Count  Last   PosInQueue                                 Key    Count  Last   PosInQueue  
--------------------------------                                --------------------------------
(0, 1) 2      3      0                                          (0, 1) 0      -1     0           
(1, 0) 1      2      0                                          (1, 0) 0      -1     0           
                                                                (0, 3) 1      2      -1          
                                                                (3, 1) 1      3      -1          

----------------------------------------------------------------------------------------------------
Sequence (before vs after)
Index  Symbol     Pos    PrevKTuple   NextKTuple            

In [4]:
#A and D - no old pairs elsewhere, no new pairs

active_k_tuples_before = {
    (0, 1): KTupleInfo(count=1, last=1, pos_in_queue=0),
    (1, 2): KTupleInfo(count=1, last=2, pos_in_queue=1),
    (2, 3): KTupleInfo(count=1, last=3, pos_in_queue=2),
}
active_k_tuples_after = {
    (0, 1): KTupleInfo(count=0, last=-1, pos_in_queue=0),
    (1, 2): KTupleInfo(count=0, last=-1, pos_in_queue=1),
    (2, 3): KTupleInfo(count=0, last=-1, pos_in_queue=2),
    (0, 4): KTupleInfo(count=1, last=2, pos_in_queue=-1),
    (4, 3): KTupleInfo(count=1, last=3, pos_in_queue=-1),
}
sequence_before = [
    SequenceElement(symbol=0, pos=0, prev_k_tuple=-1, next_k_tuple=-1),
    SequenceElement(symbol=1, pos=1, prev_k_tuple=-1, next_k_tuple=-1),
    SequenceElement(symbol=2, pos=2, prev_k_tuple=-1, next_k_tuple=-1),
    SequenceElement(symbol=3, pos=3, prev_k_tuple=-1, next_k_tuple=-1),
]
sequence_after = [
    SequenceElement(symbol=0, pos=0, prev_k_tuple=-1, next_k_tuple=-1),
    SequenceElement(symbol=-1, pos=1, prev_k_tuple=0, next_k_tuple=2),
    SequenceElement(symbol=4, pos=2, prev_k_tuple=-1, next_k_tuple=-1),
    SequenceElement(symbol=3, pos=3, prev_k_tuple=-1, next_k_tuple=-1),
]



debug("0123", (1,2), 4, active_k_tuples_before, active_k_tuples_after, sequence_before, sequence_after)
print("-------------------------------------------------------------------------------------------")
print("                                          OK                                               ")
print("-------------------------------------------------------------------------------------------")

active_k_tuples before
OK
sequence before
OK
active_k_tuples after
OK
sequence after
OK

----------------------------------------------------------------------------------------------------
Active K-Tuples (before vs after)
Key    Count  Last   PosInQueue                                 Key    Count  Last   PosInQueue  
--------------------------------                                --------------------------------
(0, 1) 1      1      0                                          (0, 1) 0      -1     0           
(1, 2) 1      2      1                                          (1, 2) 0      -1     1           
(2, 3) 1      3      2                                          (2, 3) 0      -1     2           
                                                                (0, 4) 1      2      -1          
                                                                (4, 3) 1      3      -1          

------------------------------------------------------------------------------------------

In [5]:
#A and D - old pairs to left and right


active_k_tuples_before = {
    (0, 1): KTupleInfo(count=3, last=9, pos_in_queue=0),
    (1, 2): KTupleInfo(count=2, last=10, pos_in_queue=0),
    (2, 3): KTupleInfo(count=3, last=11, pos_in_queue=1),
    (3, 2): KTupleInfo(count=1, last=4, pos_in_queue=0),
    (3, 0): KTupleInfo(count=1, last=6, pos_in_queue=1),
    (1, 0): KTupleInfo(count=1, last=8, pos_in_queue=2),
}

active_k_tuples_after = {
    (0, 1): KTupleInfo(count=2, last=9, pos_in_queue=0),
    (1, 2): KTupleInfo(count=2, last=10, pos_in_queue=0),
    (2, 3): KTupleInfo(count=2, last=11, pos_in_queue=1),
    (3, 2): KTupleInfo(count=1, last=4, pos_in_queue=0),
    (3, 0): KTupleInfo(count=0, last=-1, pos_in_queue=1),
    (1, 0): KTupleInfo(count=1, last=8, pos_in_queue=2),
    (2, 5): KTupleInfo(count=1, last=6, pos_in_queue=-1),
    (5, 1): KTupleInfo(count=1, last=7, pos_in_queue=-1),
}

sequence_before = [
    SequenceElement(symbol=0, pos=0, prev_k_tuple=-1, next_k_tuple=-1),
    SequenceElement(symbol=1, pos=1, prev_k_tuple=-1, next_k_tuple=7),
    SequenceElement(symbol=2, pos=2, prev_k_tuple=-1, next_k_tuple=10),
    SequenceElement(symbol=3, pos=3, prev_k_tuple=-1, next_k_tuple=5),
    SequenceElement(symbol=2, pos=4, prev_k_tuple=-1, next_k_tuple=-1),
    SequenceElement(symbol=3, pos=5, prev_k_tuple=3, next_k_tuple=11),
    SequenceElement(symbol=0, pos=6, prev_k_tuple=-1, next_k_tuple=-1),
    SequenceElement(symbol=1, pos=7, prev_k_tuple=1, next_k_tuple=9),
    SequenceElement(symbol=0, pos=8, prev_k_tuple=-1, next_k_tuple=-1),
    SequenceElement(symbol=1, pos=9, prev_k_tuple=7, next_k_tuple=-1),
    SequenceElement(symbol=2, pos=10, prev_k_tuple=2, next_k_tuple=-1),
    SequenceElement(symbol=3, pos=11, prev_k_tuple=5, next_k_tuple=-1),
]

sequence_after = [
    SequenceElement(symbol=0, pos=0, prev_k_tuple=-1, next_k_tuple=-1),
    SequenceElement(symbol=1, pos=1, prev_k_tuple=-1, next_k_tuple=9),
    SequenceElement(symbol=2, pos=2, prev_k_tuple=-1, next_k_tuple=10),
    SequenceElement(symbol=3, pos=3, prev_k_tuple=-1, next_k_tuple=11),
    SequenceElement(symbol=2, pos=4, prev_k_tuple=-1, next_k_tuple=-1),
    SequenceElement(symbol=-1, pos=5, prev_k_tuple=4, next_k_tuple=6),
    SequenceElement(symbol=5, pos=6, prev_k_tuple=-1, next_k_tuple=-1),
    SequenceElement(symbol=1, pos=7, prev_k_tuple=-1, next_k_tuple=-1),
    SequenceElement(symbol=0, pos=8, prev_k_tuple=-1, next_k_tuple=-1),
    SequenceElement(symbol=1, pos=9, prev_k_tuple=1, next_k_tuple=-1),
    SequenceElement(symbol=2, pos=10, prev_k_tuple=2, next_k_tuple=-1),
    SequenceElement(symbol=3, pos=11, prev_k_tuple=3, next_k_tuple=-1),
]

debug("CDABABCDCDAB", (3,0), 5, active_k_tuples_before, active_k_tuples_after, sequence_before, sequence_after)
print("-------------------------------------------------------------------------------------------")
print("                                          OK                                               ")
print("-------------------------------------------------------------------------------------------")

active_k_tuples before
OK
sequence before
OK
active_k_tuples after
OK
sequence after
OK

----------------------------------------------------------------------------------------------------
Active K-Tuples (before vs after)
Key    Count  Last   PosInQueue                                 Key    Count  Last   PosInQueue  
--------------------------------                                --------------------------------
(0, 1) 3      9      0                                          (0, 1) 2      9      0           
(1, 2) 2      10     0                                          (1, 2) 2      10     0           
(2, 3) 3      11     1                                          (2, 3) 2      11     1           
(3, 2) 1      4      0                                          (3, 2) 1      4      0           
(3, 0) 1      6      1                                          (3, 0) 0      -1     1           
(1, 0) 1      8      2                                          (1, 0) 1      8      2     

In [6]:
#first old pairs replaced 

active_k_tuples_before = {
    (0, 1): KTupleInfo(count=2, last=7, pos_in_queue=0),
    (1, 2): KTupleInfo(count=1, last=2, pos_in_queue=0),
    (2, 3): KTupleInfo(count=2, last=5, pos_in_queue=1),
    (3, 2): KTupleInfo(count=1, last=4, pos_in_queue=1),
    (3, 0): KTupleInfo(count=1, last=6, pos_in_queue=2),
}

active_k_tuples_after = {
    (0, 1): KTupleInfo(count=1, last=1, pos_in_queue=0),
    (1, 2): KTupleInfo(count=1, last=2, pos_in_queue=0),
    (2, 3): KTupleInfo(count=1, last=3, pos_in_queue=1),
    (3, 2): KTupleInfo(count=1, last=4, pos_in_queue=1),
    (3, 0): KTupleInfo(count=0, last=-1, pos_in_queue=2),
    (2, 5): KTupleInfo(count=1, last=6, pos_in_queue=-1),
    (5, 1): KTupleInfo(count=1, last=7, pos_in_queue=-1),
}

sequence_before = [
    SequenceElement(symbol=0, pos=0, prev_k_tuple=-1, next_k_tuple=-1),
    SequenceElement(symbol=1, pos=1, prev_k_tuple=-1, next_k_tuple=7),
    SequenceElement(symbol=2, pos=2, prev_k_tuple=-1, next_k_tuple=-1),
    SequenceElement(symbol=3, pos=3, prev_k_tuple=-1, next_k_tuple=5),
    SequenceElement(symbol=2, pos=4, prev_k_tuple=-1, next_k_tuple=-1),
    SequenceElement(symbol=3, pos=5, prev_k_tuple=3, next_k_tuple=-1),
    SequenceElement(symbol=0, pos=6, prev_k_tuple=-1, next_k_tuple=-1),
    SequenceElement(symbol=1, pos=7, prev_k_tuple=1, next_k_tuple=-1),
]

sequence_after = [
    SequenceElement(symbol=0, pos=0, prev_k_tuple=-1, next_k_tuple=-1),
    SequenceElement(symbol=1, pos=1, prev_k_tuple=-1, next_k_tuple=-1),
    SequenceElement(symbol=2, pos=2, prev_k_tuple=-1, next_k_tuple=-1),
    SequenceElement(symbol=3, pos=3, prev_k_tuple=-1, next_k_tuple=-1),
    SequenceElement(symbol=2, pos=4, prev_k_tuple=-1, next_k_tuple=-1),
    SequenceElement(symbol=-1, pos=5, prev_k_tuple=4, next_k_tuple=6),
    SequenceElement(symbol=5, pos=6, prev_k_tuple=-1, next_k_tuple=-1),
    SequenceElement(symbol=1, pos=7, prev_k_tuple=-1, next_k_tuple=-1),
]

debug("23010123", (3,0), 5, active_k_tuples_before, active_k_tuples_after, sequence_before, sequence_after)
print("-------------------------------------------------------------------------------------------")
print("                                          OK                                               ")
print("-------------------------------------------------------------------------------------------")

active_k_tuples before
OK
sequence before
OK
active_k_tuples after
OK
sequence after
OK

----------------------------------------------------------------------------------------------------
Active K-Tuples (before vs after)
Key    Count  Last   PosInQueue                                 Key    Count  Last   PosInQueue  
--------------------------------                                --------------------------------
(0, 1) 2      7      0                                          (0, 1) 1      1      0           
(1, 2) 1      2      0                                          (1, 2) 1      2      0           
(2, 3) 2      5      1                                          (2, 3) 1      3      1           
(3, 2) 1      4      1                                          (3, 2) 1      4      1           
(3, 0) 1      6      2                                          (3, 0) 0      -1     2           
                                                                (2, 5) 1      6      -1    

In [7]:
# ABAB - replacing BA

active_k_tuples_before = {
    (0, 1): KTupleInfo(count=2, last=3, pos_in_queue=0),
    (1, 0): KTupleInfo(count=1, last=2, pos_in_queue=0),
}

active_k_tuples_after = {
    (0, 1): KTupleInfo(count=0, last=-1, pos_in_queue=0),
    (1, 0): KTupleInfo(count=0, last=-1, pos_in_queue=0),
    (1, 2): KTupleInfo(count=0, last=-1, pos_in_queue=-1),
    (2, 2): KTupleInfo(count=1, last=3, pos_in_queue=-1),
}

sequence_before = [
    SequenceElement(symbol=0, pos=0, prev_k_tuple=-1, next_k_tuple=-1),
    SequenceElement(symbol=1, pos=1, prev_k_tuple=-1, next_k_tuple=3),
    SequenceElement(symbol=0, pos=2, prev_k_tuple=-1, next_k_tuple=-1),
    SequenceElement(symbol=1, pos=3, prev_k_tuple=1, next_k_tuple=-1),
]

sequence_after = [
    SequenceElement(symbol=-1, pos=0, prev_k_tuple=-1, next_k_tuple=1),
    SequenceElement(symbol=2, pos=1, prev_k_tuple=-1, next_k_tuple=-1),
    SequenceElement(symbol=-1, pos=2, prev_k_tuple=1, next_k_tuple=3),
    SequenceElement(symbol=2, pos=3, prev_k_tuple=-1, next_k_tuple=-1),
]


debug("0101", (0,1), 2, active_k_tuples_before, active_k_tuples_after, sequence_before, sequence_after)
print("-------------------------------------------------------------------------------------------")
print("                                          OK                                               ")
print("-------------------------------------------------------------------------------------------")
# AAA - replacing AA


active_k_tuples before
OK
sequence before
OK
active_k_tuples after
OK
sequence after
OK

----------------------------------------------------------------------------------------------------
Active K-Tuples (before vs after)
Key    Count  Last   PosInQueue                                 Key    Count  Last   PosInQueue  
--------------------------------                                --------------------------------
(0, 1) 2      3      0                                          (0, 1) 0      -1     0           
(1, 0) 1      2      0                                          (1, 0) 0      -1     0           
                                                                (1, 2) 0      -1     -1          
                                                                (2, 2) 1      3      -1          

----------------------------------------------------------------------------------------------------
Sequence (before vs after)
Index  Symbol     Pos    PrevKTuple   NextKTuple            

In [9]:
active_k_tuples_before = {
    (0, 1): KTupleInfo(count=3, last=5, pos_in_queue=0),
    (1, 0): KTupleInfo(count=2, last=4, pos_in_queue=0),
}

active_k_tuples_after = {
    (0, 1): KTupleInfo(count=0, last=-1, pos_in_queue=0),
    (1, 0): KTupleInfo(count=0, last=-1, pos_in_queue=0),
    (1, 3): KTupleInfo(count=0, last=-1, pos_in_queue=-1),
    (3, 3): KTupleInfo(count=2, last=5, pos_in_queue=-1),
}

sequence_before = [
    SequenceElement(symbol=0, pos=0, prev_k_tuple=-1, next_k_tuple=-1),
    SequenceElement(symbol=1, pos=1, prev_k_tuple=-1, next_k_tuple=3),
    SequenceElement(symbol=0, pos=2, prev_k_tuple=-1, next_k_tuple=4),
    SequenceElement(symbol=1, pos=3, prev_k_tuple=1, next_k_tuple=5),
    SequenceElement(symbol=0, pos=4, prev_k_tuple=2, next_k_tuple=-1),
    SequenceElement(symbol=1, pos=5, prev_k_tuple=3, next_k_tuple=-1),
]

sequence_after = [
    SequenceElement(symbol=-1, pos=0, prev_k_tuple=-1, next_k_tuple=1),
    SequenceElement(symbol=3, pos=1, prev_k_tuple=-1, next_k_tuple=-1),
    SequenceElement(symbol=-1, pos=2, prev_k_tuple=1, next_k_tuple=3),
    SequenceElement(symbol=3, pos=3, prev_k_tuple=-1, next_k_tuple=5),
    SequenceElement(symbol=-1, pos=4, prev_k_tuple=3, next_k_tuple=5),
    SequenceElement(symbol=3, pos=5, prev_k_tuple=3, next_k_tuple=-1),
]

debug("010101", (0,1), 3, active_k_tuples_before, active_k_tuples_after, sequence_before, sequence_after)
print("-------------------------------------------------------------------------------------------")
print("                                          OK                                               ")
print("-------------------------------------------------------------------------------------------")

active_k_tuples before
OK
sequence before
OK
active_k_tuples after
OK
sequence after
OK

----------------------------------------------------------------------------------------------------
Active K-Tuples (before vs after)
Key    Count  Last   PosInQueue                                 Key    Count  Last   PosInQueue  
--------------------------------                                --------------------------------
(0, 1) 3      5      0                                          (0, 1) 0      -1     0           
(1, 0) 2      4      0                                          (1, 0) 0      -1     0           
                                                                (1, 3) 0      -1     -1          
                                                                (3, 3) 2      5      -1          

----------------------------------------------------------------------------------------------------
Sequence (before vs after)
Index  Symbol     Pos    PrevKTuple   NextKTuple            

In [4]:
debug("000", (0,0), 3)
# we're not ready for situation where the next tuple to be processed (prev BC) dissapears
# it is worrying we have -1 in the tuple - is it a problem with the get_prev/next_index functions?

TypeError: debug() missing 4 required positional arguments: 'ab', 'aa', 'sb', and 'sa'

In [13]:
s, a = construct_active_k_tuples_and_sequence(string_to_symbol_list("0101")[0], 2)
for x in SequenceElement.get_print_lines(s): print(x)
for x in KTupleInfo.get_print_lines(a): print(x)

Index  Symbol     Pos    PrevKTuple   NextKTuple  
--------------------------------------------------
0      0          0      -1           -1          
1      1          1      -1           3           
2      0          2      -1           -1          
3      1          3      1            -1          
Key    Count  Last   PosInQueue  
--------------------------------
(0, 1) 2      3      -1          
(1, 0) 1      2      -1          


In [14]:
s, a = construct_active_k_tuples_and_sequence(string_to_symbol_list("000")[0], 2)
for x in SequenceElement.get_print_lines(s): print(x)
for x in KTupleInfo.get_print_lines(a): print(x)

Index  Symbol     Pos    PrevKTuple   NextKTuple  
--------------------------------------------------
0      0          0      -1           -1          
1      0          1      -1           2           
2      0          2      1            -1          
Key    Count  Last   PosInQueue  
--------------------------------
(0, 0) 2      2      -1          


In [67]:
s, a = construct_active_k_tuples_and_sequence(string_to_symbol_list("012012")[0], 2)
SequenceElement.print_list(s)
KTupleInfo.print_dict(a)

Index  Symbol     Pos    PrevKTuple   NextKTuple  
--------------------------------------------------
0      0          0      -1           -1          
1      1          1      -1           4           
2      2          2      -1           5           
3      0          3      -1           -1          
4      1          4      1            -1          
5      2          5      2            -1          
Key    Count  Last   PosInQueue  
--------------------------------
(0, 1) 2      4      -1          
(1, 2) 2      5      -1          
(2, 0) 1      3      -1          


In [69]:
sequence, active_k_tuples = construct_active_k_tuples_and_sequence(string_to_symbol_list("012012")[0], 2)
print("active_k_tuples before")
KTupleInfo.print_dict(active_k_tuples)
priority_queue = construct_priority_queue(active_k_tuples)
print("active_k_tuples after")
KTupleInfo.print_dict(active_k_tuples)
print("priority queue")
pprint(priority_queue)

active_k_tuples before
Key    Count  Last   PosInQueue  
--------------------------------
(0, 1) 2      4      -1          
(1, 2) 2      5      -1          
(2, 0) 1      3      -1          
active_k_tuples after
Key    Count  Last   PosInQueue  
--------------------------------
(0, 1) 2      4      0           
(1, 2) 2      5      1           
(2, 0) 1      3      0           
priority queue
{1: [(2, 0)], 2: [(0, 1), (1, 2)]}
